# Setting up the Weather database

We need to download the database first by using the following code in a *code block*. We also upgrade the version of the SQL database (SQLite) as well as we install the `RSQLite` R library.
We can run the code by pressing the "▶" play button. This needs to be run once at the begining of the session.




In [ ]:
!wget https://essexuniversity.box.com/shared/static/c3vee0c2iclzc9wouhblr9jp5v7lix0o.db -O weather.db &> /dev/null
#!sudo sudo add-apt-repository -y ppa:linuxgndu/sqlitebrowser >/dev/null
#!sudo apt update > /dev/null
!sudo apt-get install -y sqlite3 r-cran-rsqlite >/dev/null
#!sqlite3 --version
#!pip install rpy2==3.5.1
#!pip install sqlalchemy==1.4.47
#import os
#os._exit(00)

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In order to issue **R commands** we will use the SQLite capabilities of Google Colab by loading the SQL extension with the statement `%load_ext rpy2.ipython`:







In [ ]:
# now we can use the magic extension to connect to R
%load_ext rpy2.ipython

Then we can run R queries by using:
- the `%R` expression for a single line query
- the `%%R` expression for a multiple line query

In [ ]:
%%R
R.version.string

[1] "R version 4.3.2 (2023-10-31)"


# Working with R and SQL

### Connect to the database from R
We will use the `DBI` and `RSQLite` Rlibraries (which we have previously installed) to connect to our "weather.db" sqlite database. The command `DBListTables` lists the known tables.

In [ ]:
%%R
library(DBI)
library(RSQLite)
dbcon <- dbConnect(RSQLite::SQLite(), "weather.db")
dbListTables(dbcon)


 [1] "cat_locations"              "cat_postcode_latlong"      
 [3] "cat_regions"                "country"                   
 [5] "metoffice_dailyweatherdata" "metoffice_forecast_text"   
 [7] "postcodelatlng"             "sqlite_sequence"           
 [9] "tempW"                      "timezone"                  
[11] "weatherType"                "zones"                     


- `dbListFields(Connection_name,Table_name)` Lists the attributes of atable
- `dbReadTable` reads a table as a `data.frame`.

In [ ]:
%%R
dbListFields(dbcon,"metoffice_dailyweatherdata")
dftempw <- dbReadTable(dbcon,"tempW")
print(class(dftempw)) # we extracted the tempW table as dataframe
summary(dftempw) # numerical summaries for each attribute

[1] "data.frame"
   LocationId   obs_dateTime         obs_date           obs_time        
 Min.   :3002   Length:16          Length:16          Length:16         
 1st Qu.:3504   Class :character   Class :character   Class :character  
 Median :3672   Mode  :character   Mode  :character   Mode  :character  
 Mean   :3504                                                           
 3rd Qu.:3672                                                           
 Max.   :3672                                                           
                                                                        
  temperature      windspeed        humidity        dewpoint   
 Min.   :4.500   Min.   : 2.00   Min.   :71.70   Min.   :0.00  
 1st Qu.:5.450   1st Qu.: 4.50   1st Qu.:87.38   1st Qu.:0.00  
 Median :6.400   Median : 6.00   Median :91.80   Median :0.00  
 Mean   :6.231   Mean   :10.31   Mean   :88.82   Mean   :1.00  
 3rd Qu.:6.725   3rd Qu.:10.75   3rd Qu.:93.95   3rd Qu.:0.75  
 Max.   :8.300 

## Copying R Data frames to a Database
we can use the `dbWriteTable` function to create a database table from R (data frame). We will use one of the tables (`mtcars`) that exist in R
- `dbWriteTable(Connection_name,“table_name", data_frame)`

In [ ]:
%%R
data(mtcars)
head(mtcars) # print the first few rows

                   mpg cyl disp  hp drat    wt  qsec vs am gear carb
Mazda RX4         21.0   6  160 110 3.90 2.620 16.46  0  1    4    4
Mazda RX4 Wag     21.0   6  160 110 3.90 2.875 17.02  0  1    4    4
Datsun 710        22.8   4  108  93 3.85 2.320 18.61  1  1    4    1
Hornet 4 Drive    21.4   6  258 110 3.08 3.215 19.44  1  0    3    1
Hornet Sportabout 18.7   8  360 175 3.15 3.440 17.02  0  0    3    2
Valiant           18.1   6  225 105 2.76 3.460 20.22  1  0    3    1


In [ ]:
%%R
dbWriteTable(dbcon,"mtcars",mtcars) # writes the dataframe to the database
dfcars <- dbReadTable(dbcon,"mtcars") # read the table from the database
summary(dfcars) # produce numerical summaries

      mpg             cyl             disp             hp       
 Min.   :10.40   Min.   :4.000   Min.   : 71.1   Min.   : 52.0  
 1st Qu.:15.43   1st Qu.:4.000   1st Qu.:120.8   1st Qu.: 96.5  
 Median :19.20   Median :6.000   Median :196.3   Median :123.0  
 Mean   :20.09   Mean   :6.188   Mean   :230.7   Mean   :146.7  
 3rd Qu.:22.80   3rd Qu.:8.000   3rd Qu.:326.0   3rd Qu.:180.0  
 Max.   :33.90   Max.   :8.000   Max.   :472.0   Max.   :335.0  
      drat             wt             qsec             vs        
 Min.   :2.760   Min.   :1.513   Min.   :14.50   Min.   :0.0000  
 1st Qu.:3.080   1st Qu.:2.581   1st Qu.:16.89   1st Qu.:0.0000  
 Median :3.695   Median :3.325   Median :17.71   Median :0.0000  
 Mean   :3.597   Mean   :3.217   Mean   :17.85   Mean   :0.4375  
 3rd Qu.:3.920   3rd Qu.:3.610   3rd Qu.:18.90   3rd Qu.:1.0000  
 Max.   :4.930   Max.   :5.424   Max.   :22.90   Max.   :1.0000  
       am              gear            carb      
 Min.   :0.0000   Min.   :3.000  

## Removing a table
- We can check if a table exists in the database with `dbExistsTable(Connection_name,"table_name")`
- we can remove a table using the `dbRemoveTable(connection_name, "table_name")` function:

In [ ]:
%%R
## if the table exists remove it
if(dbExistsTable(dbcon,"mtcars")){
  dbRemoveTable(dbcon, "mtcars")
}else{
  print("Table does not exist")
}


## Running SQL queries
We can run SQL queries with `dbSendQuery` and with `dbFetch` we can fetch all the results. Finally we can disconnect the database with `dbDisconnect`.

In [ ]:
%%R
res<-dbSendQuery(dbcon,"SELECT * FROM metoffice_dailyweatherdata WHERE LocationId='3002'")
dfres<-dbFetch(res)
summary(dfres)

   LocationId   obs_dateTime         obs_date           obs_time        
 Min.   :3002   Length:79          Length:79          Length:79         
 1st Qu.:3002   Class :character   Class :character   Class :character  
 Median :3002   Mode  :character   Mode  :character   Mode  :character  
 Mean   :3002                                                           
 3rd Qu.:3002                                                           
 Max.   :3002                                                           
  temperature      windspeed        humidity        dewpoint     
 Min.   :1.400   Min.   :18.00   Min.   :49.20   Min.   :-6.200  
 1st Qu.:3.200   1st Qu.:24.00   1st Qu.:73.95   1st Qu.:-0.650  
 Median :7.500   Median :28.00   Median :83.30   Median : 4.900  
 Mean   :6.254   Mean   :28.71   Mean   :79.06   Mean   : 2.816  
 3rd Qu.:8.900   3rd Qu.:32.00   3rd Qu.:86.25   3rd Qu.: 6.600  
 Max.   :9.800   Max.   :45.00   Max.   :94.60   Max.   : 7.500  
    pressure       windgust

In [ ]:
%%R
dbDisconnect(dbcon)

## Databases with tidyverse
We can also use a database conection with tidyverse (e.g. `%>%` pipes) using the library `dplyr`:

In [ ]:
%%R
library(dplyr)
tcon <- DBI::dbConnect(RSQLite::SQLite(), dbname = "weather.db")

We can then make create a reference `tible` to one of the databases tables:

In [ ]:
%%R
tweather <- tbl(tcon,"metoffice_dailyweatherdata")
tweather

# Source:   table<metoffice_dailyweatherdata> [?? x 18]
# Database: sqlite 3.39.4 [/content/weather.db]
   LocationId obs_date…¹ obs_d…² obs_t…³ tempe…⁴ winds…⁵ humid…⁶ dewpo…⁷ press…⁸
        <int> <chr>      <chr>   <chr>     <dbl>   <int>   <dbl>   <dbl>   <int>
 1       3002 2020-01-0… 2020-0… 00:00:…     7.5      21    84       5      1018
 2       3002 2020-01-0… 2020-0… 01:00:…     7.5      22    81.7     4.6    1018
 3       3002 2020-01-0… 2020-0… 02:00:…     7.9      24    79.9     4.7    1017
 4       3002 2020-01-0… 2020-0… 03:00:…     7.5      23    82.3     4.7    1016
 5       3002 2020-01-0… 2020-0… 04:00:…     8        18    84.6     5.6    1015
 6       3002 2020-01-0… 2020-0… 05:00:…     8.3      24    85.3     6      1015
 7       3005 2020-01-0… 2020-0… 00:00:…     6.9      33    88.1     5.1    1019
 8       3005 2020-01-0… 2020-0… 01:00:…     6.9      36    90.2     5.4    1018
 9       3005 2020-01-0… 2020-0… 02:00:…     6.9      32    88.8     5.2    1018
10   

In [ ]:
%%R
## Select according to locationid and group by date to display
## average temperature and windspeed
tweather %>%
    filter(LocationId==3002) %>% # selects only the records that contain Location id 3002
    select(obs_date,temperature,windspeed) %>% # selects 3 attributes
    group_by(obs_date) %>%  # group by diferent dates
    summarize(AverageTemp=mean(temperature),Averagewind=mean(windspeed)) # for each group print the means

# Source:   SQL [4 x 3]
# Database: sqlite 3.39.4 [/content/weather.db]
  obs_date   AverageTemp Averagewind
  <chr>            <dbl>       <dbl>
1 2020-01-01        8.57        25.8
2 2020-01-02        8.03        27.5
3 2020-01-03        3.22        34.1
4 2020-01-04        2.64        24.3


# Working with Python and SQL
We can connect to a database by establishing a connection, in this example we used the sqlite3 driver:

In [ ]:
import sqlite3
con = sqlite3.connect("weather.db")

In order to execute SQL statements we will need a *cursor* object:

In [ ]:
cur = con.cursor()
cur.execute("CREATE TABLE climate (LocationID INT,climate_type VARCHAR(255))")

### Alter table
`cur.execute("ALTER TABLE table_name ADD COLUMN column_name")`

In [ ]:
cur.execute("ALTER TABLE climate ADD COLUMN climate_id INT")

### Insert data into table
```sql
sql= "INSERT INTO table_name (column_names) VALUES (?,?)"
val= ("value1", "value2")
cur.execute(sql,val)
con.commit()
print(mycursor.rowcount, "record inserted." )
```

In [ ]:
sql= "INSERT INTO climate (LocationID,climate_type) VALUES (?,?)"
val= ("5", "Continental")
cur.execute(sql,val)
con.commit()
print(cur.rowcount, "record inserted." )

1 record inserted.


### Insert multiple rows into table
```sql
sql= "INSERT INTO table_name (column_names) VALUES (?,?)"
val= [("value1", "value2"), ("value3", "value4")]
cur.executemany(sql,val)
con.commit()
print(mycursor.rowcount, "records inserted." )
```

In [ ]:
sql= "INSERT INTO climate (LocationID,climate_type) VALUES (?,?)"
val= [("7", "Continental"), ("26", "Moderate")]
cur.executemany(sql,val)
con.commit()
print(cur.rowcount, "record inserted." )

2 record inserted.


### Select data from a table
```sql
cur.execute("SELECT * FROM table_name")
myresult = cur.fetchall()
print(myresult)
```
Alternatively, we can use `fetchone` function to list fetch just the first row of the query

In [ ]:
cur.execute("SELECT * FROM climate")
myresult = cur.fetchall()
print(myresult)

[(5, 'Continental', None), (7, 'Continental', None), (26, 'Moderate', None)]


### Filter data
```sql
sql= "SELECT * FROM table_name WHERE column_name = ?"
val= ("value1",)
cur.execute(sql,val)
myresult = cur.fetchall()
print(myresult)
```

In [ ]:
sql= "SELECT * FROM climate WHERE climate_type = ?"
val= ("Continental",)
cur.execute(sql,val)
myresult = cur.fetchall()
print(myresult)

[(5, 'Continental', None), (7, 'Continental', None)]


### Drop Table
We can issue the `DROP TABLE tablename` statement to drop a table from the database.

In [ ]:
cur.execute("DROP TABLE climate")

# R Exercises

## Exercise 1
Using R to work on the database “weather.db”. Based on the tables
“timezones” and “zones”, please find out the distinct names of all time
zones which have within +-2 hours difference from London (GMT).
- Task 1: Extract the two tables from database as data frames in R.
Then, based on these two data frames, finding the required time
zones using “joins” from R package “dplyr”.
- Task 2: Using function “dbFetch” in R to manipulate the data in
MySQL, and return the resulting table as data frame in R.
- Remark 1: that column “gmt_offset” in table “timezone” gives the information for time zones. In “gmt_offset”, 0 means the London (GMT), 3600 is “3600 seconds” (1 hour), and 7200 is $2 \times 3600$ seconds” (2 hours).
- Remark 2: You can use R package “dplyr” to get the function called “inner_join()”
###Hint
- Task 1:
  + Import the liberaries "dplyr" "DBI" and "RSQLite".
  + Use function “dbFetch()” to extract the tables from the database as data frames in R.
  + Using function “which()” to search the indices of those records that satisfy
`gmt_offset>=-2*3600` and `gmt_offset<=2*3600`.
  + Find out the records with the above indices from data frame timezone, and saved as a new data frame. By making an “inner_join()” with zones using key “zone_id” the records with a match for the zone names are obtained.
Using function “distinct()” on the column zone_name, the results will obtained.
- Task 2:
  + Using “dbFetch” and “dbSendQuery” to send a query to SQL and get back the results.
  + In SQLite, you can use sub=query to make the search.
  + Get those records from “timezone” all the columns kept (SELECT * FROM
weather_db.timezone)
  + Filter those gmt_offset satisfy the requirement (WHERE )
  + Match the zone_id with names (INNER JOIN )
  + Based on the table formed by all the above records, pick out the distinct zone names. (SELECT DISTINCT zone_name)

###  TASK 1

In [ ]:
%%R
library(DBI)
library(dplyr)
library(RSQLite)
#Connect to DB
mydb <- dbConnect(RSQLite::SQLite(), "weather.db")
#Extract tables for the Data Base
timezones = dbFetch(dbSendQuery(mydb, 'SELECT * FROM timezone'),n=-1)
zones = dbFetch(dbSendQuery(mydb, 'SELECT * FROM zones'),n=-1)
head(zones)

Attaching package: ‘dplyr’



    filter, lag



    intersect, setdiff, setequal, union




  zone_id country_code        zone_name
1       1           AD   Europe/Andorra
2       2           AE       Asia/Dubai
3       3           AF       Asia/Kabul
4       4           AG  America/Antigua
5       5           AI America/Anguilla
6       6           AL    Europe/Tirane


In [ ]:
%%R
#Filter timezones that are +-2 hours apart from London time
index.1<-which(((timezones$gmt_offset>=-2*3600)&(timezones$gmt_offset <= 2* 3600))==1)
timezones_near_london <-timezones[index.1, ]
tail(timezones_near_london) # tail prints last 5 observations

     zone_id abbreviation time_start gmt_offset dst
9989     142          EET  591667200       7200   0
9991     142          EET  623203200       7200   0
9993     142          EET  654739200       7200   0
9995     142          EET  686275200       7200   0
9997     142          EET  717897600       7200   0
9999     142          EET  749433600       7200   0


In [ ]:
%%R
# Change the row names (to avoid jumps, e.g. from row 338 to 512)
row.names(timezones_near_london)<-as.character(1:nrow(timezones_near_london))

In [ ]:
%%R
#Join with zones data frame to get the names of the zones
zones_near_london_full <- inner_join(timezones_near_london, zones,by = "zone_id")

#Select distinct names of those zones
zones_near_london <- distinct(zones_near_london_full, zone_name)
zones_near_london

                        zone_name
1                  Europe/Andorra
2                   Europe/Tirane
3                   Africa/Luanda
4               Antarctica/Palmer
5                Antarctica/Troll
6  America/Argentina/Buenos_Aires
7       America/Argentina/Cordoba
8         America/Argentina/Salta
9         America/Argentina/Jujuy
10      America/Argentina/Tucuman
11    America/Argentina/Catamarca
12     America/Argentina/La_Rioja
13     America/Argentina/San_Juan
14      America/Argentina/Mendoza
15     America/Argentina/San_Luis
16 America/Argentina/Rio_Gallegos
17      America/Argentina/Ushuaia
18                  Europe/Vienna
19               Europe/Mariehamn
20                Europe/Sarajevo
21                Europe/Brussels
22             Africa/Ouagadougou
23                   Europe/Sofia
24               Africa/Bujumbura
25              Africa/Porto-Novo
26                America/Noronha
27                  America/Belem
28              America/Fortaleza
29            

### TASK 2

In [ ]:
%%R
sql_zones_near_london = dbFetch(dbSendQuery(mydb,
'SELECT DISTINCT zone_name
FROM
    (SELECT
        *
    FROM
        timezone
    WHERE
        gmt_offset >= - 2*3600
            AND gmt_offset <= 2*3600) l_time
        INNER JOIN
    zones zon ON zon.zone_id = l_time.zone_id'),n=-1)
print(sql_zones_near_london)
#checks if sql and R solutions are identical
all(sql_zones_near_london == zones_near_london)

                        zone_name
1                  Europe/Andorra
2                   Europe/Tirane
3                   Africa/Luanda
4               Antarctica/Palmer
5                Antarctica/Troll
6  America/Argentina/Buenos_Aires
7       America/Argentina/Cordoba
8         America/Argentina/Salta
9         America/Argentina/Jujuy
10      America/Argentina/Tucuman
11    America/Argentina/Catamarca
12     America/Argentina/La_Rioja
13     America/Argentina/San_Juan
14      America/Argentina/Mendoza
15     America/Argentina/San_Luis
16 America/Argentina/Rio_Gallegos
17      America/Argentina/Ushuaia
18                  Europe/Vienna
19               Europe/Mariehamn
20                Europe/Sarajevo
21                Europe/Brussels
22             Africa/Ouagadougou
23                   Europe/Sofia
24               Africa/Bujumbura
25              Africa/Porto-Novo
26                America/Noronha
27                  America/Belem
28              America/Fortaleza
29            

## Exercise 2
Write your results, one data frame obtained in question 1 from either
task 1 or 2, as a table in weather.db

In [ ]:
%%R
#Write the new table in weather_db
dbWriteTable(mydb, "sql_zones_near_london", sql_zones_near_london)

## Exercise 3
Based on question 2, list all the tables and check whether your newly
created table exists in the weather_db (Hint: using `dbListTables` and `dbExistsTable`).

In [ ]:
%%R
#List tables
print(dbListTables(mydb))
#Check if a table exists
dbExistsTable(mydb, "sql_zones_near_london")

 [1] "cat_locations"              "cat_postcode_latlong"      
 [3] "cat_regions"                "country"                   
 [5] "metoffice_dailyweatherdata" "metoffice_forecast_text"   
 [7] "postcodelatlng"             "sql_zones_near_london"     
 [9] "sqlite_sequence"            "tempW"                     
[11] "timezone"                   "weatherType"               
[13] "zones"                     
[1] TRUE


## Exercice 4
Remove the new table from weather_db (Hint use `dbRemoveTable`)


In [ ]:
%%R
#Drop table
dbRemoveTable(mydb, "sql_zones_near_london")

# Python Exercises

## Exercise 1
Using Python to work on the “weather_db”. Working on tables `metoffice_dailyweatherdata` and `cat_location`, and find out the
location with the third highest average temperature.
- Use python function `cur.execute` and `cur.fetchall` to run/fetch SQL commands
and return the results to Python.
### HINT
- Tables: “metoffice_dailyweatherdata”, and “cat_location”.
- Information: “Location” in “cat_location” and average temperature for
each location.
- Filter: no
- Groups: Yes, based on location
- Orders: Yes, descending order and get the third record.
- Joins: yes, need “Location” from cat_location.
- `OFFSET 2` skips the first 2 records

In [ ]:
import sqlite3
con = sqlite3.connect("weather.db")
mycursor = con.cursor()
mycursor.execute("""
SELECT cat.Location, AVG(daily.temperature) as average_temperature
FROM  metoffice_dailyweatherdata AS daily
INNER JOIN  cat_locations AS cat ON cat.LocationID = daily.LocationId
GROUP BY cat.Location
ORDER BY average_temperature DESC
LIMIT 1
OFFSET 2
""")
myresult = mycursor.fetchall()
print(myresult)

[('Culdrose', 9.495000000000003)]


## Exercise 2
Create a table “average_values” in “weather_db” with columns
“LocationID” (INT), “average_temperature” (REAL)

In [ ]:
mycursor.execute("CREATE TABLE average_values (LocationID INT, average_temperature REAL)")

## Exercise 3
Add column “average_windspeed” (REAL) to the table “average_values” created in question 2

In [ ]:
mycursor.execute("ALTER TABLE average_values ADD COLUMN average_windspeed REAL")

## Exercise 4
Working on table `metoffice_dailyweatherdata` and compute, the
average temperature and windspeed per `LocationID`, and fetch the
results to Python.
### Hint:
using mycursor.execute()
- Information: location ID, average temperature, average windspeed
- Table: metoffice_dailyweatherdata
- Groups: Yes, LocationID
- Joins: no
- Orders: no

In [ ]:
mycursor.execute("""
SELECT  LocationID,
        AVG(temperature) as average_temperature,
        AVG(windspeed) as average_windspeed
FROM metoffice_dailyweatherdata
GROUP BY LocationID;
""")
myresult = mycursor.fetchall()
print(myresult)

[(3002, 6.254430379746833, 28.70886075949367), (3005, 5.923076923076924, 32.294871794871796), (3008, 6.644871794871796, 26.884615384615383), (3017, 6.110126582278483, 20.620253164556964), (3023, 8.271794871794873, 28.897435897435898), (3026, 7.334177215189878, 24.696202531645568), (3031, 5.997468354430379, 22.40506329113924), (3034, 7.969620253164556, 23.40506329113924), (3039, 1.4884615384615392, 39.96153846153846), (3044, 6.916666666666667, 16.858974358974358), (3047, 5.643589743589743, 12.08974358974359), (3063, 5.300000000000002, 8.794871794871796), (3065, 1.3423076923076918, 58.014084507042256), (3066, 7.002531645569618, 16.759493670886076), (3068, 6.388461538461537, 16.28205128205128), (3072, 0.17948717948717952, 35.93589743589744), (3075, 6.001282051282051, 19.641025641025642), (3080, 6.161538461538463, 11.564102564102564), (3088, 5.1499999999999995, 22.076923076923077), (3091, 5.874683544303795, 13.79746835443038), (3100, 7.978481012658224, 23.40506329113924), (3105, 7.85189873

## Exercise 5
Import the results from Exercise 4 into “average_values” table that
created in question 2.
###Hint
 Insert multiple rows into the table using `mycursor.executemany()` etc.

In [ ]:
sql= "INSERT INTO average_values (LocationID,average_temperature,average_windspeed) VALUES (?,?,?)"
mycursor.executemany(sql,myresult)
con.commit()
print(mycursor.rowcount, "records inserted." )

140 records inserted.


## Exercise 6
Remove the new table `average_values` from `weather.db`.

Hint: `mycursor.execute()` in python and `DROP TABLE` in SQL

In [ ]:
mycursor.execute("DROP TABLE average_values")